In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import Bunch
# ExplainableBoostingClassifier
from interpret.glassbox import ExplainableBoostingClassifier
import lightgbm as lgb

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, recall_score, balanced_accuracy_score

from sklearn.preprocessing import TargetEncoder

In [2]:
# Define the chunk size for reading the CSV file
chunksize = 100000  # Adjust this value based on your requirements

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Read the CSV file in chunks based on the defined chunk size
for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
    # Filter the current chunk based on the criteria
    filtered_chunk = chunk[chunk['issue_d'].str.contains("2010", na=False)]
    # filtered_chunk = filtered_chunk[~filtered_chunk['issue_d'].str.contains("Oct-2013|Nov-2013|Dec-2013", na=False)]
    # Append the filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df = pd.concat(filtered_chunks)


/tmp/ipykernel_12145/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_12145/2038679434.py:8: DtypeWarning: Columns (19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_12145/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_12145/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Dat

In [3]:
df_1 = df.sample(frac=1, random_state=42)
df_1.shape

(12537, 151)

In [4]:
import re

def preprocess_data(df):
    # Calculate risk score
    df["risk_score"] = df.loc[:, ["last_fico_range_high", "last_fico_range_low"]].mean(axis=1)
    
    # Create target variable
    df["target"] = np.where((df.loan_status == 'Current') |
                            (df.loan_status == 'Fully Paid') |
                            (df.loan_status == "Issued") |
                            (df.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 0, 1)
    
    # Drop unnecessary columns
    for c in ['last_fico_range_high', 'last_fico_range_low', 'loan_status']:
        df = df.drop(c, axis=1, errors='ignore')
    
    def fix_term_column(df):
        """
        Fix the dtype of the 'term' column by extracting the numeric part and converting it to an integer.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'term' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'term' column converted to numeric.
        """
        try:
            # Extract numeric part from the 'term' column and convert to integer
            df['term'] = df['term'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if pd.notna(x) else x)
        except AttributeError as e:
            # Handle cases where the regex search fails (e.g., no match found)
            print("Error: Unable to extract numeric value from 'term' column.")
            print(f"Details: {e}")
        except Exception as e:
            # Handle any other unexpected errors
            print("An unexpected error occurred while fixing the 'term' column.")
            print(f"Details: {e}")
        
        return df
    
    def fix_emp_length_column(df):
        """
        Fix the dtype of the 'emp_length' column by converting it to numeric.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'emp_length' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'emp_length' column converted to numeric.
        """
        try:
            # Map '< 1 year' to '0' and extract numeric part from the 'emp_length' column
            df['emp_length'] = df['emp_length'].map(lambda x: "0" if x == '< 1 year' else x)
            df['emp_length'] = df['emp_length'].map(lambda x: int(re.search(r'\d+', str(x)).group()), na_action='ignore')
        except Exception as e:
            # Handle any unexpected errors
            print("Error: Unable to convert 'emp_length' column to numeric.")
            print(f"Details: {e}")
        
        return df
    
    df = fix_term_column(df)
    df = fix_emp_length_column(df)

    return df

In [5]:
df_2 = preprocess_data(df_1)

In [6]:
drop_list = [
    'issue_d', 'url', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d', 'pymnt_plan', 'recoveries', 
    'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'hardship_flag', 
    'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 
    'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 
    'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 
    'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 
    'settlement_percentage', 'settlement_term', 'id', 'emp_title', 'desc', 'title', 'last_credit_pull_d', 'next_pymnt_d',
]

In [7]:
def calculate_earliest_cr_line_days(df, column_name="earliest_cr_line"):
    """
    Calculate the number of days since 1900 based on the 'earliest_cr_line' column.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The name of the column to process. Default is 'earliest_cr_line'.

    Returns:
        pd.Series: A Series containing the calculated days.
    """
    month_dict = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }

    years_in_days = df[column_name].map(lambda x: int(x[-4:])).map(lambda x: (2025 - x) * 365)
    total_days = years_in_days + df[column_name].map(lambda x: x[:3]).replace(month_dict).map(lambda x: 365 - (x * 30))
    
    return total_days

In [8]:
df_2 = preprocess_data(df_1)
# df_2['earliest_cr_line'] = calculate_earliest_cr_line_days(df_2)
# df_2['earliest_cr_line']

In [9]:
X = df_2.loc[:, df_2.columns != 'target'].drop(drop_list, axis=1).dropna(axis = 1, inplace=False).drop('risk_score', axis = 1)
y = df_2['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
ebm = ExplainableBoostingClassifier(random_state=42)
ebm.fit(X_train, y_train)

ExplainableBoostingClassifier()

In [11]:
ebm_pred_proba = ebm.predict_proba(X_test)[:, 1]
ebm_pred = ebm.predict(X_test)

In [12]:
lgbm = make_pipeline(TargetEncoder(), StandardScaler(), lgb.LGBMClassifier(random_state=42))
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1408, number of negative: 8621
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001726 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3332
[LightGBM] [Info] Number of data points in the train set: 10029, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.140393 -> initscore=-1.812031
[LightGBM] [Info] Start training from score -1.812031


Pipeline(steps=[('targetencoder', TargetEncoder()),
                ('standardscaler', StandardScaler()),
                ('lgbmclassifier', LGBMClassifier(random_state=42))])

In [13]:
lgbm_pred_proba = lgbm.predict_proba(X_test)[:, 1]
lgbm_pred = lgbm.predict(X_test)

/home/athyrson/Code/py10/lib64/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/athyrson/Code/py10/lib64/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [14]:
#evaulate both models and present them in a df
metrics = pd.DataFrame({
    'Model': ['EBM', 'LGBM'],
    'ROC AUC': [roc_auc_score(y_test, ebm_pred_proba), roc_auc_score(y_test, lgbm_pred_proba)],
    'Recall': [recall_score(y_test, ebm_pred), recall_score(y_test, lgbm_pred)],
    'Balanced Accuracy': [balanced_accuracy_score(y_test, ebm_pred), balanced_accuracy_score(y_test, lgbm_pred)],
})

metrics

,Model,ROC AUC,Recall,Balanced Accuracy
0,EBM,0.695394,0.037249,0.515382
1,LGBM,0.673068,0.028653,0.509695


In [15]:
eg = ebm.explain_global()
eg.selector

,Name,Type,# Unique,% Non-zero
0,loan_amnt,continuous,377.0,NaN
1,funded_amnt,continuous,642.0,NaN
2,funded_amnt_inv,continuous,3102.0,NaN
3,term,continuous,2.0,NaN
4,int_rate,continuous,117.0,NaN
...,...,...,...,...
64,fico_range_low & inq_last_6mths,interaction,NaN,NaN
65,fico_range_high & inq_last_6mths,interaction,NaN,NaN
66,inq_last_6mths & open_acc,interaction,NaN,NaN
67,inq_last_6mths & revol_bal,interaction,NaN,NaN


In [16]:
from interpret import show
show(ebm.explain_global())

<!-- http://127.0.0.1:7170/140312335874848/ -->

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.datasets import load_iris



make_pipeline(TargetEncoder(), StandardScaler(), lgb.LGBMClassifier(random_state=42))
# Compute mutual information (which measures entropy reduction)
feature_importance = mutual_info_classif(X_train, y_train, discrete_features=False)

# Convert to a DataFrame for better readability
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Information Gain': feature_importance})
importance_df = importance_df.sort_values(by='Information Gain', ascending=False)

# Print feature importance based on entropy reduction
print(importance_df)


ValueError: could not convert string to float: 'D'